In [1]:
import pandas as pd
import numpy as np
import sys
import seaborn as sns
import os.path
import time, itertools, re
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, make_scorer, classification_report
import textwrap
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, train_test_split,StratifiedGroupKFold
from imblearn.over_sampling import SMOTENC, SMOTE#, make_pipeline
#from imblearn.pipeline import make_pipeline
from sklearn.pipeline import make_pipeline, Pipeline
#procs_path='/'.join(os.getcwd().split('/')[0:4])+ '/mimiciv/hosp_icu/'
from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
print(os.getcwd())
fileloc_data='/'.join(os.getcwd().split('/')[0:5])+ '/data/annonymizedDatasets/'
print(fileloc_data)
#############################################
df_main=pd.read_csv(fileloc_data+'maskedDAIsy_main.tsv', sep='\t', decimal=',')
print(df_main.columns)
print(df_main['pid'].nunique(), df_main['intid'].nunique())
df_main.head(3)

/home/sreejita/ProjectsPy/GGZ/code_scripts
/home/sreejita/ProjectsPy/GGZ/data/annonymizedDatasets/
Index(['intid', 'pid', 'p_startdate', 'p_dischargedate', 'age', 'bsex',
       'edu_level', 'edu_comp', 'Highest_Edu', 'EDtype', 'icd9_code',
       'icd9_lab'],
      dtype='object')
969 1008


,intid,pid,p_startdate,p_dischargedate,age,bsex,edu_level,edu_comp,Highest_Edu,EDtype,icd9_code,icd9_lab
0,3,408,2133-02-16,2134-10-16,31,Vrouw,NaN,NaN,Onbekend,Anorexia nervosa,307.1,Anorexia nervosa: Beperkende type
1,4,1095,2133-01-16,2133-09-25,34,Vrouw,NaN,NaN,Onbekend,Other ED,307.5,Eetstoornis NAO
2,5,1479,2133-02-04,2135-09-24,15,Vrouw,NaN,NaN,Onbekend,Others,NaN,NaN


In [2]:
df_dectool_fu=pd.read_csv(fileloc_data+'maskedDAIsy_DECTOOLS_followups.tsv', sep='\t', decimal=',')
print(df_dectool_fu['pid'].nunique(),df_dectool_fu['intid'].nunique())
df_dectool_first=pd.read_csv(fileloc_data+'maskedDAIsy_DECTOOLS_first_inst.tsv', sep='\t', decimal=',')
print(df_dectool_first['pid'].nunique(),df_dectool_first['intid'].nunique())
main_dectool_pids=np.intersect1d(df_dectool_first['intid'].unique(),df_main['intid'].unique())
print(len(main_dectool_pids))
cols2include=np.setdiff1d(df_dectool_first.columns, ['pid'])
pid_per_intid=df_dectool_first['pid'].value_counts()
print(pid_per_intid)
print('PIDs repeated in INTIDs: ', pid_per_intid[pid_per_intid.values>1])
#df_dectool_first.head(4)

9 9
225 227
223
pid
530.0     2
461.0     2
1065.0    1
1071.0    1
1078.0    1
         ..
587.0     1
593.0     1
596.0     1
604.0     1
497.0     1
Name: count, Length: 225, dtype: int64
PIDs repeated in INTIDs:  pid
530.0    2
461.0    2
Name: count, dtype: int64


In [3]:
df_main_dectools=df_main.merge(df_dectool_first[cols2include], on='intid', how='inner')
print(df_main_dectools.shape)
print(df_main['age'].min(), df_main['age'].max(), df_main['age'].median())
print(df_main_dectools['age'].min(), df_main_dectools['age'].max(), df_main_dectools['age'].median())
print(df_main_dectools['pid'].nunique(), df_main_dectools['intid'].nunique())
pid_per_intid=df_main_dectools['pid'].value_counts()
#print(pid_per_intid)
print('PIDs repeated in INTIDs: ', pid_per_intid[pid_per_intid.values>1])
print(df_main_dectools['EDtype'].value_counts())
df_main_dectools.head(3)

(223, 31)
12 66 24.0
17 65 26.0
222 223
PIDs repeated in INTIDs:  pid
530    2
Name: count, dtype: int64
EDtype
Anorexia nervosa    83
Bulimia nervosa     41
Binge-ED            40
Other ED            37
Others              22
Name: count, dtype: int64


,intid,pid,p_startdate,p_dischargedate,age,bsex,edu_level,edu_comp,Highest_Edu,EDtype,...,IND_ocd_comorbiditeit,aantal_eerdere_trajecten,andere_comorbiditeiten,duur_stoornis_in_jaren,interpretation,leeftijd_invullen,p_dectool_invultijd,uitslag_waarde,verstoord_eetpatroon,volgorde
0,794,56,2138-09-25,2143-09-12,52,Vrouw,"HBO bachelor (HBO, HTS diploma)",HBO,HBO bachelor,Other ED,...,0,2.0,PTSS,40,Geïndiceerd voor hoogspecialistische zorg op b...,55,2141-07-28 15:03:34,4,NaN,2
1,850,1406,2139-02-01,2141-12-31,17,Vrouw,"HBO bachelor (HBO, HTS diploma)",HBO,HBO bachelor,Bulimia nervosa,...,0,1.0,NaN,3,Geïndiceerd voor hoogspecialistische zorg op b...,19,2141-04-07 15:24:55,4,NaN,1
2,993,340,2139-11-25,2143-12-07,54,Vrouw,"HBO bachelor (HBO, HTS diploma)",HBO,HBO bachelor,Anorexia nervosa,...,0,4.0,"angststoornis, trauma, pers. stoornis",25,Geïndiceerd voor hoogspecialistische zorg op b...,57,2142-11-01 22:36:27,4,Voornamelijk vloeibaar en beperkt,1


In [4]:
df_edeq_all=pd.read_csv(fileloc_data+'maskedDAIsy_EDEQ_Score.tsv', sep='\t', decimal=',')
df_edeq_all['seq_num'] = df_edeq_all.groupby('intid').cumcount()+1
df_edeq_all.to_csv(fileloc_data+'maskedDAIsy_EDEQ_Score_SeqNum.tsv', sep='\t', decimal=',', index=False)
df_edeq_first=df_edeq_all.loc[df_edeq_all['seq_num']==1]
df_edeq_second=df_edeq_all.loc[df_edeq_all['seq_num']==2]
df_edeq_third=df_edeq_all.loc[df_edeq_all['seq_num']==3]
df_edeq_fourth=df_edeq_all.loc[df_edeq_all['seq_num']==4]
df_mdect_ed_first=df_edeq_first[['intid', 'seq_num','EDEQscore']].merge(df_main_dectools, on='intid', how='inner')
df_mdect_ed_second=df_edeq_second[['intid', 'seq_num','EDEQscore']].merge(df_main_dectools, on='intid', how='inner')
df_mdect_ed_third=df_edeq_third[['intid', 'seq_num','EDEQscore']].merge(df_main_dectools, on='intid', how='inner')
df_mdect_ed_fourth=df_edeq_fourth[['intid', 'seq_num','EDEQscore']].merge(df_main_dectools, on='intid', how='inner')
print(df_mdect_ed_first['EDEQscore'].min(), df_mdect_ed_first['EDEQscore'].median(), df_mdect_ed_first['EDEQscore'].max())
print(df_mdect_ed_second['EDEQscore'].min(), df_mdect_ed_second['EDEQscore'].median(), df_mdect_ed_second['EDEQscore'].max())
ranges=[0.0,1.5,3,4.5,6]
print(df_mdect_ed_first.loc[df_mdect_ed_first['intid'].isin(df_mdect_ed_first['intid'].unique())].groupby(
    pd.cut(df_mdect_ed_first.EDEQscore, ranges))['pid'].count())
print('First visit only:\n',df_mdect_ed_first.groupby(['EDtype'])['intid'].count(), df_mdect_ed_first.shape)
print('Second visit only:\n',df_mdect_ed_second.groupby(['EDtype'])['intid'].count(), df_mdect_ed_second.shape)
print('Third visit only:\n',df_mdect_ed_third.groupby(['EDtype'])['intid'].count(), df_mdect_ed_third.shape)
print('Fourth visit only:\n',df_mdect_ed_fourth.groupby(['EDtype'])['intid'].count(), df_mdect_ed_fourth.shape)
#df_edeq_third.to_csv(fileloc_data+'maskedDAIsy_MainDectoolEDEQ_Visit3_Only.tsv', sep='\t', decimal=',', index=False)

0.0 4.03 5.95
0.0 3.84 5.62
EDEQscore
(0.0, 1.5]    12
(1.5, 3.0]    43
(3.0, 4.5]    97
(4.5, 6.0]    70
Name: pid, dtype: int64
First visit only:
 EDtype
Anorexia nervosa    83
Binge-ED            40
Bulimia nervosa     41
Other ED            37
Others              22
Name: intid, dtype: int64 (223, 33)
Second visit only:
 EDtype
Anorexia nervosa    83
Binge-ED            40
Bulimia nervosa     41
Other ED            37
Others              22
Name: intid, dtype: int64 (223, 33)
Third visit only:
 EDtype
Anorexia nervosa    47
Binge-ED            24
Bulimia nervosa     26
Other ED            21
Others              15
Name: intid, dtype: int64 (133, 33)
Fourth visit only:
 EDtype
Anorexia nervosa    32
Binge-ED            14
Bulimia nervosa     13
Other ED            13
Others              12
Name: intid, dtype: int64 (84, 33)


## Train and test splits and checking the class distribution in train and test
### Train and test sets created such that both have at similar distribution of patients present in till at least second follow-up and all other patients (only 1 visit, only 1 follow-up, all three follow-ups).

In [5]:
label_2fus=df_mdect_ed_third['EDtype'].to_list()
feature_list=np.setdiff1d(df_main_dectools.columns, ['EDtype'])
Xtrain_2followups, Xtest_2followups, ytrain_2fu, ytest_2fu = train_test_split(df_mdect_ed_third[feature_list], label_2fus, 
                                                      test_size=0.25, random_state=42, stratify=label_2fus)
df_mdect_ed_rest=df_mdect_ed_first.loc[~df_mdect_ed_first['intid'].isin(df_mdect_ed_third['intid'].unique())].copy()
label_rest=df_mdect_ed_rest['EDtype'].to_list()
Xtrain_rest, Xtest_rest, ytrain_rest, ytest_rest = train_test_split(df_mdect_ed_rest[feature_list], label_rest, 
                                                      test_size=0.25, random_state=42, stratify=label_rest)
h_tr=np.union1d(Xtrain_2followups['intid'].astype(float).unique(), Xtrain_rest['intid'].astype(float).unique())
h_te=np.union1d(Xtest_2followups['intid'].astype(float).unique(), Xtest_rest['intid'].astype(float).unique())
print(len(h_tr), len(h_te))
print(df_mdect_ed_first['EDtype'].loc[df_mdect_ed_first['intid'].isin(h_tr)].value_counts())
print(df_mdect_ed_first['EDtype'].loc[df_mdect_ed_first['intid'].isin(h_te)].value_counts())

df_mdect_ed_first['Split']='Train'
df_mdect_ed_first['Split'].loc[df_mdect_ed_first['intid'].astype(float).isin(h_te)]='Test'
df_mdect_ed_second['Split']='Train'
df_mdect_ed_second['Split'].loc[df_mdect_ed_second['intid'].astype(float).isin(h_te)]='Test'
df_mdect_ed_third['Split']='Train'
df_mdect_ed_third['Split'].loc[df_mdect_ed_third['intid'].astype(float).isin(h_te)]='Test'
df_mdect_ed_fourth['Split']='Train'
df_mdect_ed_fourth['Split'].loc[df_mdect_ed_fourth['intid'].astype(float).isin(h_te)]='Test'

df_mdect_ed_first.to_csv(fileloc_data+'maskedDAIsy_MainDectoolEDEQ_Visit1_Only.tsv', sep='\t', decimal=',', index=False)
df_mdect_ed_second.to_csv(fileloc_data+'maskedDAIsy_MainDectoolEDEQ_Visit2_Only.tsv', sep='\t', decimal=',', index=False)
df_mdect_ed_third.to_csv(fileloc_data+'maskedDAIsy_MainDectoolEDEQ_Visit3_Only.tsv', sep='\t', decimal=',', index=False)
df_mdect_ed_fourth.to_csv(fileloc_data+'maskedDAIsy_MainDectoolEDEQ_Visit4_Only.tsv', sep='\t', decimal=',', index=False)

166 57
EDtype
Anorexia nervosa    62
Binge-ED            30
Bulimia nervosa     30
Other ED            28
Others              16
Name: count, dtype: int64
EDtype
Anorexia nervosa    21
Bulimia nervosa     11
Binge-ED            10
Other ED             9
Others               6
Name: count, dtype: int64


In [4]:
df_sq48=pd.read_csv(fileloc_data+'maskedDAIsy_SQ48_all.tsv', sep='\t', decimal=',')
freq_sq48=df_sq48['intid'].value_counts()
focus_intids_fusq48=np.array(freq_sq48[freq_sq48.values>=2].keys().astype(float))
df_sq48_fu=df_sq48.loc[df_sq48['intid'].astype(float).isin(focus_intids_fusq48)].copy()

print(df_main_dectools['EDtype'].loc[df_main_dectools['pid'].astype(float).isin(df_sq48_fu['pid'].unique())].value_counts())
print(df_main_dectools['EDtype'].loc[df_main_dectools['pid'].astype(float).isin(df_sq48['pid'].unique())].value_counts())
df_sq48.head(4)

EDtype
Anorexia nervosa    87
Bulimia nervosa     43
Binge-ED            39
Other ED            37
Others              24
Name: count, dtype: int64
EDtype
Anorexia nervosa    90
Bulimia nervosa     44
Binge-ED            40
Other ED            38
Others              24
Name: count, dtype: int64


,pid,intid,p_sq48_filled_date,Vijandigheid,Agorafobie,Angst,Cognitieve_Klachten,Depressie,Somatische_Klachten,Sociale_Fobie,Totaalscore,Vitaliteit_Optimisme,Werk_Studie
0,0,1491,2142-11-17,4.0,2.0,13.0,14.0,9.0,14.0,16.0,72.0,13.0,0.0
1,0,1491,2143-05-16,4.0,3.0,12.0,14.0,6.0,13.0,16.0,68.0,10.0,0.0
2,0,1491,2143-09-19,8.0,1.0,15.0,13.0,9.0,15.0,11.0,72.0,12.0,8.0
3,0,1491,2143-11-17,4.0,1.0,13.0,14.0,5.0,13.0,14.0,64.0,10.0,8.0


In [5]:
cols2include=np.setdiff1d(df_main_dectools.columns, ['intid'])
df_main_sq48=df_sq48.merge(df_main_dectools[cols2include], on='pid', how='inner')
print(df_main_sq48.columns)
#cols_main_sq48=['pid', 'p_startdate', 'p_dischargedate', 'age', 'bsex', 'Highest_Edu', 'EDtype', 'icd9_code','icd9_lab',
#        'intid_y', 'p_sq48_filled_date', 'Vijandigheid', 'Agorafobie', 'Angst', 'Cognitieve_Klachten', 'Depressie',
#       'Somatische_Klachten', 'Sociale_Fobie', 'Totaalscore','Vitaliteit_Optimisme', 'Werk_Studie']
#df_main_sq48=df_main_sq48[cols_main_sq]
df_main_sq48.head(4)

Index(['pid', 'intid', 'p_sq48_filled_date', 'Vijandigheid', 'Agorafobie',
       'Angst', 'Cognitieve_Klachten', 'Depressie', 'Somatische_Klachten',
       'Sociale_Fobie', 'Totaalscore', 'Vitaliteit_Optimisme', 'Werk_Studie',
       'BMI', 'EDtype', 'Highest_Edu', 'IND_anders',
       'IND_as_I_II_comorbiditeiten', 'IND_borderline_comorbiditeit',
       'IND_depressie_comorbiditeit', 'IND_duur_stoornis_2plus_jaar',
       'IND_eerdere_spec_behandeling_zonder_effect',
       'IND_ernstig_verstoord_eten', 'IND_extreem_laag_hoog_BMI',
       'IND_ocd_comorbiditeit', 'aantal_eerdere_trajecten', 'age',
       'andere_comorbiditeiten', 'bsex', 'duur_stoornis_in_jaren', 'edu_comp',
       'edu_level', 'icd9_code', 'icd9_lab', 'interpretation',
       'leeftijd_invullen', 'p_dectool_invultijd', 'p_dischargedate',
       'p_startdate', 'uitslag_waarde', 'verstoord_eetpatroon', 'volgorde'],
      dtype='object')


,pid,intid,p_sq48_filled_date,Vijandigheid,Agorafobie,Angst,Cognitieve_Klachten,Depressie,Somatische_Klachten,Sociale_Fobie,...,icd9_code,icd9_lab,interpretation,leeftijd_invullen,p_dectool_invultijd,p_dischargedate,p_startdate,uitslag_waarde,verstoord_eetpatroon,volgorde
0,0,1491,2142-11-17,4.0,2.0,13.0,14.0,9.0,14.0,16.0,...,307.51,"Eetbuistoornis: matig, ernstig, zeer ernstig",Geïndiceerd voor hoogspecialistische zorg op b...,47,2142-12-14 00:00:00,NaN,2143-03-08,3,NaN,1
1,0,1491,2143-05-16,4.0,3.0,12.0,14.0,6.0,13.0,16.0,...,307.51,"Eetbuistoornis: matig, ernstig, zeer ernstig",Geïndiceerd voor hoogspecialistische zorg op b...,47,2142-12-14 00:00:00,NaN,2143-03-08,3,NaN,1
2,0,1491,2143-09-19,8.0,1.0,15.0,13.0,9.0,15.0,11.0,...,307.51,"Eetbuistoornis: matig, ernstig, zeer ernstig",Geïndiceerd voor hoogspecialistische zorg op b...,47,2142-12-14 00:00:00,NaN,2143-03-08,3,NaN,1
3,0,1491,2143-11-17,4.0,1.0,13.0,14.0,5.0,13.0,14.0,...,307.51,"Eetbuistoornis: matig, ernstig, zeer ernstig",Geïndiceerd voor hoogspecialistische zorg op b...,47,2142-12-14 00:00:00,NaN,2143-03-08,3,NaN,1


In [13]:
df_mhcsf=pd.read_csv(fileloc_data+'maskedDAIsy_mhcsf.tsv', sep='\t', decimal=',')
freq_mhcsf=df_mhcsf['intid'].value_counts()
focus_intids_fumhcsf=np.array(freq_mhcsf[freq_mhcsf.values>=2].keys().astype(float))
df_mhcsf_fu=df_mhcsf.loc[df_mhcsf['intid'].astype(float).isin(focus_intids_fumhcsf)].copy()

df_main_mhcsf=df_mhcsf.merge(df_main_dectools[cols2include], on='pid', how='inner')

print(df_main_dectools['EDtype'].loc[df_main_dectools['pid'].astype(float).isin(df_mhcsf_fu['pid'].unique())].value_counts())
print(df_main_dectools['EDtype'].loc[df_main_dectools['pid'].astype(float).isin(df_mhcsf['pid'].unique())].value_counts())
df_mhcsf.head(4)

EDtype
Anorexia nervosa    86
Bulimia nervosa     42
Binge-ED            39
Other ED            36
Others              24
Name: count, dtype: int64
EDtype
Anorexia nervosa    90
Bulimia nervosa     44
Binge-ED            39
Other ED            38
Others              24
Name: count, dtype: int64


,intid,pid,p_mhcsf_filled_date,EmotionWB,SocialWB,PsychWB,mhcsf_score
0,1491,0,2142-11-17,3.3,2.0,2.5,2.5
1,1491,0,2143-05-16,3.7,2.0,2.0,2.4
2,1491,0,2143-09-19,3.3,2.6,2.3,2.6
3,1491,0,2143-11-17,3.3,2.4,2.3,2.6


In [10]:
print(df_main['EDtype'].loc[df_main['pid'].astype(float).isin(df_dectool_fu['pid'].unique())].value_counts())

print(df_main['EDtype'].loc[df_main['pid'].astype(float).isin(df_honos_fu['pid'].unique())].value_counts())

print(df_main['EDtype'].value_counts())

EDtype
Other ED            5
Anorexia nervosa    3
Binge-ED            2
Name: count, dtype: int64
EDtype
Anorexia nervosa    20
Other ED             9
Others               9
Binge-ED             7
Bulimia nervosa      1
Name: count, dtype: int64
EDtype
Anorexia nervosa    571
Others              335
Other ED            314
Bulimia nervosa     252
Binge-ED            134
Name: count, dtype: int64


In [14]:
print(df_sq48['pid'].nunique(), df_mhcsf['pid'].nunique(), 
      len(np.intersect1d(df_sq48['pid'].unique(), df_mhcsf['pid'].unique())),
      len(np.intersect1d(df_main_dectools['pid'].unique(), df_mhcsf['pid'].unique())),
      len(np.intersect1d(df_sq48['pid'].unique(), df_main_dectools['pid'].unique())),
     len(np.intersect1d(df_main_dectools['pid'].unique(),np.intersect1d(df_sq48['pid'].unique(), df_mhcsf['pid'].unique()))))


433 426 425 222 223 222


In [15]:
print('Negative: ', df_sqmhs['intid'].loc[df_sqmhs['DiffToMHCSF']<0].nunique())
print('Around a year: ', df_sqmhs['intid'].loc[(df_sqmhs['DiffToMHCSF'].abs()-365.25)<=40].nunique())
print('More than one year', df_sqmhs['intid'].loc[(df_sqmhs['DiffToMHCSF']>=0) & ((df_sqmhs['DiffToMHCSF'].abs()-365)>40)].nunique())
print('Negative: ', df_sqmhs['intid'].loc[df_sqmhs['DiffToMHCSF']<0].nunique())
print('Around a year: ', df_sqmhs['intid'].loc[(df_sqmhs['DiffToMHCSF'].abs()-365.25)<=40].nunique())
print('More than one year', df_sqmhs['intid'].loc[(df_sqmhs['DiffToMHCSF']>=0) & ((df_sqmhs['DiffToMHCSF'].abs()-365)>40)].nunique())


Negative:  465
Around a year:  544
More than one year 36


In [4]:
df_honos=pd.read_csv(fileloc_data+'maskedDAIsy_honos.tsv', sep='\t', decimal=',')
print(df_honos['pid'].nunique(),df_honos['intid'].nunique())
df_honos_first=pd.read_csv(fileloc_data+'maskedDAIsy_honos_first_inst.tsv', sep='\t', decimal=',')
print(df_honos_first['pid'].nunique(),df_honos_first['intid'].nunique())
freq_honos=df_honos['intid'].value_counts()
focus_intids_fu=np.array(freq_honos[freq_honos.values>=2].keys().astype(float))
df_honos_fu=df_honos.loc[df_honos['intid'].astype(float).isin(focus_intids_fu)].copy()
print(len(np.intersect1d(df_main['pid'].astype(float), df_honos_first['pid'].astype(float))))
print(len(np.intersect1d(df_main['pid'].astype(float), df_honos_fu['pid'].astype(float))))
df_honos_first.head(4)

227 173
174 173
174
35


,pid,intid,questionnaire_key,p_honos_filled_date,Somscore,Beperkingen,Functioneren,Gedragsproblemen,Symptomalogie,SocialeProblemen,p_honos_filled_date.1,p_honos_send_date
0,0,1491.0,honos_plus,2142-12-15,17,4,13.0,0,4,9,2142-12-15,2143-03-27
1,1,NaN,honos_plus,2142-03-01,15,3,11.0,2,4,6,2142-03-01,2142-10-08
2,15,1402.0,honos_plus,2142-07-05,13,4,8.0,2,5,2,2142-07-05,2142-10-08
3,25,1294.0,honos_plus,2142-03-24,14,2,10.0,4,4,4,2142-03-24,2142-10-13


In [ ]:
cols_to_include=['age','SOMscore', 'Disabilities', 'Functional', 'Behavioural_Prob', 'Symptom', 'Social_Prob', 'sex']
X=df_main[cols_to_include].copy()
Y1,Y2=df_main['EDtype'].to_list(), df_main['EDEQscore'].to_list()
X_train, X_test, y_train, y_test = train_test_split(X, Y1, test_size=0.20, random_state=42, stratify=Y1)

train_mean_cont=np.nanmean(X_train[['age','SOMscore', 'Disabilities', 'Functional', 'Behavioural_Prob', 'Symptom', 'Social_Prob']].to_numpy(), axis=0)
train_std_cont=np.std(X_train[['age', 'SOMscore', 'Disabilities', 'Functional', 'Behavioural_Prob', 'Symptom', 'Social_Prob']].to_numpy(), axis=0)
train_mean=train_mean_cont
train_mean.append(0)#np.insert(train_mean_cont, 0,0)#=np.concatenate((0,train_mean_cont), axis=0)
train_std=train_std_cont
train_std.append(1)#(train_std_cont, 0,1)#=np.concatenate((0,train_std_cont), axis=0)
train_mean

In [ ]:
print(X_train.isnull().sum())
print(train_mean)

In [ ]:
#df_main['HighestEdu']=''
#levels=['WO','Universitair + . Opleiding tot graad van doctor, arts, apotheker', 'HBO', 'MBO', 'VWO', 'HAVO', 'VMBO',
#        'Speciaal basis onderwijs','LO/LVO, LBO', 'Bassisschool']
#for level in levels:
    #df_main['HighestEdu'].loc[(df_main['HighestEdu']=='') & (df_main['c_edu_level'].str.contains(level, regex=True,  na=False))]=level
#df_main['HighestEdu'].loc[df_main['HighestEdu']=='']='Onbekend'
#df_main.head(5)
def gmm_bic_score(estimator, X):
    """Callable to pass to GridSearchCV that will use the BIC score."""
    # Make it negative since GridSearchCV expects a score to maximize
    return -estimator.bic(X)

In [ ]:
#X_train, X_val, y_train, y_val = train_test_split(X_train2, y_train2, test_size=0.10, random_state=42, stratify=y_train2)
x_train=(X_train-train_mean)/train_std.T
#x_val=(X_val-train_mean.T)/train_std.T

param_grid = {
    "n_components": range(3, 8),
    "covariance_type": ["diag", "full",]# "spherical"],
    #"init_params":['kmeans','k-means++', 'random_from_data']
}
grid_search = GridSearchCV(GaussianMixture(n_init=5, max_iter=300), param_grid=param_grid, scoring=gmm_bic_score)
grid_search.fit(x_train)
#=np.nanmean(X_train2.to_numpy(), axis=0), np.nanstd(X_train2.to_numpy(), axis=0)
df = pd.DataFrame(grid_search.cv_results_)[
    ["param_n_components", "param_covariance_type",# "param_init_params", 
     "mean_test_score", ]
]
df["mean_test_score"] = -df["mean_test_score"]
df = df.rename(
    columns={        
        "param_n_components": "Number of components",
        "param_covariance_type": "Type of covariance",
        "param_init_params": "Init params",
        "mean_test_score": "BIC score",
    }
)
sns.catplot(data=df, kind="bar", x="Number of components", y="BIC score", hue="Type of covariance",)
plt.show()
df.sort_values(by=["BIC score"]).head()

In [ ]:
x_train.isnull().sum()

In [ ]:
X_train